In [2]:
from rlboard import *

width, height = 8,8
m = Board(width,height)
m.randomize(seed=13)
m.plot()

error: OpenCV(4.7.0) D:\a\opencv-python\opencv-python\opencv\modules\imgproc\src\color.cpp:182: error: (-215:Assertion failed) !_src.empty() in function 'cv::cvtColor'


In [ ]:
import gym
import tensorflow as tf
from tensorflow.keras import layers, models

env = gym.make('CartPole-v1')
num_actions = env.action_space.n

model = models.Sequential([
    layers.Dense(24, input_shape=env.observation_space.shape, activation='relu'),
    layers.Dense(24, activation='relu'),
    layers.Dense(num_actions, activation='linear')
])

model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001), loss='mse')

def epsilon_greedy_policy(state, epsilon=0.1):
    if np.random.rand() < epsilon:
        return np.random.randint(num_actions)
    return np.argmax(model.predict(state[np.newaxis]))

# Training the DQN
num_episodes = 1000
batch_size = 32
gamma = 0.99

for episode in range(num_episodes):
    state = env.reset()
    state = np.reshape(state, [1, env.observation_space.shape[0]])
    done = False

    while not done:
        action = epsilon_greedy_policy(state)
        next_state, reward, done, _ = env.step(action)
        next_state = np.reshape(next_state, [1, env.observation_space.shape[0]])
        target = reward + gamma * np.amax(model.predict(next_state))
        target_f = model.predict(state)
        target_f[0][action] = target
        model.fit(state, target_f, epochs=1, verbose=0)
        state = next_state

env.close()
